In [1]:
from kubernetes import client, config

def are_all_pods_ready(namespace):
    config.load_kube_config()  # Use load_incluster_config() if running inside a pod

    # Create an instance of the API class
    v1 = client.CoreV1Api()
    try:
        # List all Pods in the specified namespace
        pods = v1.list_namespaced_pod(namespace)
        
        for pod in pods.items:
            # Check if the pod status is not 'Running' or if any of the containers are not ready
            if pod.status.phase != 'Running' or not all(container.ready for container in pod.status.container_statuses):
                return False  # Return False if any pod is not ready
        
        return True  # Return True if all pods are ready
    except Exception as e:
        print(f"An error occurred: {e}")
        return False  # Return False in case of any error during API call

# Example usage
namespace = "default"  # Specify the namespace you want to check
if are_all_pods_ready(namespace):
    print("All pods in the namespace are ready and running.")
else:
    print("Some pods in the namespace are not ready or not running.")


All pods in the namespace are ready and running.


In [6]:
import pandas as pd

initial_placement = pd.read_csv('/home/ubuntu/ms_scheduling/book_info/pod_placement/initial_pod_placement_bookinfo_v2.csv')
# Filter the DataFrame for the deployment "reviews-v1"
reviews_v1_row = initial_placement[initial_placement['Deployment Name'] == 'reviews-v1']

# Select the "Host Node" value for "reviews-v1"
host_node_for_reviews_v1 = reviews_v1_row['Host Node'].values[0]

print("The host node for deployment 'reviews-v1' is:", host_node_for_reviews_v1)

,Deployment Name,Pod Name,Host Node,IP Address
0,details-v1,details-v1-5f4d584748-qxkjj,k8s-worker-8,192.168.231.237
1,example-deployment,example-deployment-b8dd999c9-wlfdk,k8s-worker-6,192.168.227.149
2,Unknown,loki-0,k8s-worker-5,192.168.18.194
3,productpage-v1,productpage-v1-564d4686f-vm58t,k8s-worker-2,192.168.140.55
4,ratings-v1,ratings-v1-686ccfb5d8-hnhzx,k8s-worker-9,192.168.251.120
5,reviews-v1,reviews-v1-56c6569848-t29jd,k8s-worker-3,192.168.69.236
6,reviews-v2,reviews-v2-847489d944-hnl6q,k8s-worker-7,192.168.5.39
7,reviews-v3,reviews-v3-768d79f845-9gw9b,k8s-worker-6,192.168.227.145


In [8]:
import sys
sys.path.append('/home/ubuntu/ms_scheduling/book_info/pod_placement')
import pods_migration

# Define the patch variables 
namespace = 'default'  # 
deployment_name = 'example-deployment'
new_node_name = 'k8s-worker-7'  # Target node for the new pods

if pods_migration.patch_deployment(deployment_name, namespace, new_node_name):
    pods_migration.wait_for_rolling_update_to_complete(deployment_name, namespace, new_node_name)

Deployment 'example-deployment' patched to schedule pods on 'k8s-worker-7'.
Waiting for the rolling update to complete...
all_pods_updated= True
len(pods)= 0
All pods are running on the new node.


In [11]:
#the pods that need to be migrated:
pod_node_binding = {'reviews-v1':'k8s-worker-2', 'reviews-v2':'k8s-worker-2','reviews-v3':'k8s-worker-2'}

Reqs_Num = [10,20,50]

for i in Reqs_Num:
    print (i)




10
20
50


In [1]:
import curl_response_time_analysis

curl_response_time_analysis.run_curl(Req_num=10)

Starting script at Wed Mar 27 12:30:44 2024


Sending Requests: 100%|██████████| 10/10 [00:33<00:00,  3.33s/request]


Script finished and plot saved at Wed Mar 27 12:31:17 2024


In [ ]:
# get the critical path from correlation matrix
descheduling_placement= {}

# migrare the pods (roll-update)
